# Test of ground station acceleration contribution in NEAR's ΔV


In [1]:
from astropy import units as u
from astropy import constants as const
from astropy import visualization
from astropy.coordinates import solar_system_ephemeris

from poliastro.util import norm
from poliastro.twobody.sampling import EpochsArray
from poliastro.frames import Planes
from poliastro.ephem import Ephem
from poliastro.bodies import Earth

import numpy as np
import sys
sys.path.append('../')

import matplotlib.pyplot as plt
import matplotlib.dates as mdates

from sim.stations import dss25, dss34
from sim.tracking import Tracking
from sim.util import orbit_from_horizons, make_epochs
from sim.util import describe_orbit, describe_trajectory, plot_residual, plot_swings
from sim.fitorbit import OrbitFitter

def pc(est_value, ref_value):
    return (est_value/ref_value - 1)*100

In [2]:
solar_system_ephemeris.set("de440")
goldstone_end = Tracking.NEAR_GOLDSTONE_END.value
canberra_start = Tracking.NEAR_CANBERRA_START.value
ref_dv = 13.46*u.mm/u.s # From Anderson et al, PRL 2008
lag_dv = 13.64*u.mm/u.s # Using lags computed from Horizons range, range rate data in NAECON 2019.

For reference, Horizons range, range rate data yield
- 3.0243 mm/s at 06:14 UTC and 2.6660 mm/s at 06:15 UTC for LOS, and
- 10.9062 mm/s at 09:53 UTC and 10.9356 mm/s at 09:54 UTC for AOS.

Antreasian & Guinn 1998 state LOS 1h 8m before perigee and AOS 2h 31m after perigee, and perigee at 07:23:55.6 UTC.
Interpolating to 06:13:55.6 UTC for LOS and 09:53:55.6 UTC for AOS led to 2.51 mm/s and 11.13 mm/s, respectively, as described in the NAECON 2019 paper.


In [3]:
e_lag = pc(lag_dv, ref_dv)
print(lag_dv, f'{e_lag:.4s}%')

13.64 mm / s 1.33%


Obtain trajectory segments from Horizons.


In [4]:
los_epochs = make_epochs(goldstone_end - 10*u.s, goldstone_end + 10*u.s, 0.5*u.s)
los_ephem = Ephem.from_horizons("NEAR", los_epochs, attractor=Earth, plane=Planes.EARTH_EQUATOR)

aos_epochs = make_epochs(canberra_start - 10*u.s, canberra_start + 10*u.s, 0.5*u.s)
aos_ephem = Ephem.from_horizons("NEAR", aos_epochs, attractor=Earth, plane=Planes.EARTH_EQUATOR)

In [5]:
# Compute range, range rate, radial acceleration and station part of acceleration

los_r, los_rr, los_ra, los_rs = dss25.range_rate_accel(los_ephem, goldstone_end)
aos_r, aos_rr, aos_ra, aos_rs = dss34.range_rate_accel(aos_ephem, canberra_start)

print(goldstone_end, los_ra << (u.m/(u.s*u.s)), los_rs << (u.m/(u.s*u.s)))
print(canberra_start, aos_ra << (u.m/(u.s*u.s)), aos_rs << (u.m/(u.s*u.s)))

1998-01-23 06:14:55.600 -0.027674608484257394 m / s2 -0.031709906465295035 m / s2
1998-01-23 09:53:55.600 -0.04551624354378624 m / s2 0.01193769052510163 m / s2


In [6]:
# Compute LOS, AOS ΔV from total radial acceleration and range 

los_dv = -los_ra*los_r/const.c << u.mm/u.s
aos_dv = -aos_ra*aos_r/const.c << u.mm/u.s
dv = los_dv + aos_dv
e_dv = pc(dv, ref_dv)
print(los_dv, aos_dv, dv, f'{e_dv:.4s}%')

2.91487741535309 mm / s 11.10567724843294 mm / s 14.02055466378603 mm / s 4.16%


In [7]:
# Recompute LOS, AOS ΔV excluding the station part
# The station part must be added, because the total acceleration is spacecraft's minus station's.

los_dvx = -(los_ra + los_rs)*los_r/const.c << u.mm/u.s
aos_dvx = -(aos_ra + aos_rs)*aos_r/const.c << u.mm/u.s
dvx = los_dvx + aos_dvx
e_dvx = pc(dvx, ref_dv)
print(los_dvx, aos_dvx, dvx, f'{e_dvx:.4s}%')

6.254779775714437 mm / s 8.192955816667212 mm / s 14.447735592381648 mm / s 7.33%
